# Using Non Earth dataset

Starting with version 3.0, rio-tiler can work with non-earth based dataset (e.g **Mars** data). In this notebook we'll show how to read such dataset and how to create a simple `non-earth` Tile server.

# Requirements

To be able to run this notebook you'll need the following requirements:
- rio-tiler~=7.0
- ipyleaflet
- matplotlib
- tornado

In [1]:
# !pip install rio-tiler
# !pip install ipyleaflet
# !pip install matplotlib
# !pip install tornado

In [2]:
from matplotlib.pyplot import imshow

# In order to fully work, we'll need to build a custom TileMatrixSet
from morecantile import TileMatrixSet
from pyproj import CRS

from rio_tiler.io import Reader

# For this DEMO we will use this file
src_path = "https://raw.githubusercontent.com/cogeotiff/rio-tiler/main/tests/fixtures/cog_nonearth.tif"

In [ ]:
# Let's first try with default
with Reader(src_path) as src:
    print(src.info().model_dump_json())

In [ ]:
# Create a CUSTOM TMS using the europa ESRI:104915 projection
europa_crs = CRS.from_authority("ESRI", 104915)
europa_tms = TileMatrixSet.custom(
    crs=europa_crs,
    extent=europa_crs.area_of_use.bounds,
    matrix_scale=[2, 1],
)

# Use Custom TMS instead of Web Mercator
with Reader(src_path, tms=europa_tms) as src:
    print(src.info().model_dump_json())
    img = src.preview()
    imshow(img.data_as_image())

In [ ]:
# Read a Tile
from rasterio.warp import transform_bounds

with Reader(src_path, tms=europa_tms) as src:
    # get dataset bounds in TMS's CRS projection
    bounds_in_tms = transform_bounds(src.crs, europa_tms.rasterio_crs, *src.bounds)
    tile = src.tms._tile(bounds_in_tms[0], bounds_in_tms[1], src.minzoom)
    print(tile)

    img = src.tile(tile.x, tile.y, tile.z)
    imshow(img.data_as_image())

## Tile Server

#### Requirements
- tornado

In [7]:
from ipyleaflet import Map, TileLayer, projections

In [8]:
from concurrent import futures

from morecantile import TileMatrixSet
from pyproj import CRS
from tornado import gen, web
from tornado.concurrent import run_on_executor
from tornado.httpserver import HTTPServer

from rio_tiler.errors import TileOutsideBounds
from rio_tiler.io import Reader
from rio_tiler.profiles import img_profiles

# Create a CUSTOM TMS using the europa ESRI:104915 projection
europa_crs = CRS.from_authority("ESRI", 104915)
europa_tms = TileMatrixSet.custom(
    crs=europa_crs,
    extent=europa_crs.area_of_use.bounds,
    matrix_scale=[2, 1],
)


class TileServer:
    def __init__(self, src_path):
        """Initialize Tornado app."""
        self.server = None
        self.app = web.Application(
            [
                (r"^/tiles/(\d+)/(\d+)/(\d+)", TileHandler, {"url": src_path}),
            ]
        )

    def start(self):
        """Start tile server."""
        self.server = HTTPServer(self.app)
        self.server.listen(8080)

    def stop(self):
        """Stop tile server."""
        if self.server:
            self.server.stop()


class TileHandler(web.RequestHandler):
    """Tile requests handler."""

    executor = futures.ThreadPoolExecutor(max_workers=16)

    def initialize(self, url):
        """Initialize tiles handler."""
        self.url = url

    @run_on_executor
    def _get_tile(self, z, x, y):

        try:
            with Reader(self.url, tms=europa_tms) as src:
                data = src.tile(x, y, z)
        except TileOutsideBounds:
            raise web.HTTPError(404)

        image = data.post_process(in_range=((0, 0.5),))

        prof = img_profiles.get("PNG", {})
        return image.render(img_format="PNG", **prof)

    @gen.coroutine
    def get(self, z, x, y):
        """Retunrs tile data and header."""
        self.set_header("Access-Control-Allow-Origin", "*")
        self.set_header("Access-Control-Allow-Methods", "GET")
        self.set_header("Content-Type", "image/png")
        self.set_header("Cache-Control", "no-store, no-cache, must-revalidate")
        res = yield self._get_tile(int(z), int(x), int(y))
        self.write(res)


ts = TileServer(src_path)
ts.start()

In [ ]:
bounds = (129.36834223297478, 13.985559117409744, 138.90253908503576, 23.13673177454536)

m = Map(
    center=((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2),
    zoom=4,
    basemap={},
    crs=projections.EPSG4326,  # HACK: the europa TMS is in degree and covers -180, -90, 180, 90 like the WGS84
)

layer = TileLayer(
    url="http://127.0.0.1:8080/tiles/{z}/{x}/{y}",
    min_zoom=4,
    max_zoom=6,
    opacity=1,
)
m.add_layer(layer)
m

In [11]:
ts.stop()